Investigating using Convolutional Networks on Weak Lensing data
=============

Adapted from 4_conv_WL


In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import time
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
from astropy.io import fits
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline
from IPython import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MultipleLocator

In [ ]:
# Reorganize the code a bit by putting the function definitions first.
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

def getFITS(imagename):
    filename = whereami + '/' + path + imagename
    f = fits.open(filename)
    dataout = f[0].data
    
    return dataout

def read_WL(path,display=None):
    # this is a version to look at sigma8
    labels=['750', '850']
    imgs = np.zeros([2048/degrade, 2048/degrade, nct, len(labels)])
    for j, label in enumerate(labels):
        for i in range(nct):
            filename = whereami + '/' + path + 'smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.'+label+'_4096xy_000'+ np.str(i+1) +'r_0029p_0100z_og.gre.fit'
            if display: print("i: %d  j: %d  name: %s" % (i, j, 'smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.'+label+'_4096xy_000'+ np.str(i+1) +'r_0029p_0100z_og.gre.fit'))
            f = fits.open(filename)
            imgs[:,:,i,j]=rebin(f[0].data, [2048/degrade, 2048/degrade])
            
    return imgs, labels

def slice_data(data, labels, exp_cut, exp_nshift):
    labels=['750', '850']
    # how many panels across
    npanelx = 2**exp_cut
    # and how big are they?
    panelw = 2048/(degrade*npanelx)
    # how many shifted panels?
    nshift = 2**exp_nshift -1
    # and what are the shifts?
    shiftw =  panelw/2**exp_nshift
    # with 4 rotations, and 2 shifts, we have
    imgs = np.zeros([panelw, panelw, nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    # let's figure out where the centers are, and save that data
    x_centers = np.zeros([nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    y_centers = np.zeros([nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    for j, label in enumerate(labels):
        for i in range(nct):
            q=0
            for k in range(npanelx):
                for l in range(npanelx):
                    for r in range(4):
                        imgs[:,:,i,q,j] = np.rot90(data[panelw*k:panelw*(k+1),panelw*l:panelw*(l+1),i, j], r)
                        x_centers[i,q,j] = (panelw*k+panelw*(k+1))/2.
                        y_centers[i,q,j] = (panelw*l+panelw*(l+1))/2.
                        q+=1
                        imgs[:,:,i,q,j] = np.fliplr(np.rot90(data[panelw*k:panelw*(k+1),panelw*l:panelw*(l+1),i, j], r))
                        x_centers[i,q,j] = (panelw*k+panelw*(k+1))/2.
                        y_centers[i,q,j] = (panelw*l+panelw*(l+1))/2.
                        q+=1
            for k in range(npanelx-1):
                for l in range(npanelx-1):
                    for m in range(nshift):
                        for n in range(nshift):
                            for r in range(4):
                                imgs[:,:,i,q,j] = np.rot90(data[panelw*k+m*shiftw:panelw*(k+1)+m*shiftw,panelw*l+n*shiftw:panelw*(l+1)+n*shiftw,i, j], r)
                                x_centers[i,q,j] = (panelw*k+m*shiftw+panelw*(k+1)+m*shiftw)/2.
                                y_centers[i,q,j] = (panelw*l+n*shiftw+panelw*(l+1)+n*shiftw)/2.
                                q+=1
                                imgs[:,:,i,q,j] = np.fliplr(np.rot90(data[panelw*k+m*shiftw:panelw*(k+1)+m*shiftw,panelw*l+n*shiftw:panelw*(l+1)+n*shiftw,i, j], r))
                                x_centers[i,q,j] = (panelw*k+m*shiftw+panelw*(k+1)+m*shiftw)/2.
                                y_centers[i,q,j] = (panelw*l+n*shiftw+panelw*(l+1)+n*shiftw)/2.
                                q+=1
    return imgs, x_centers, y_centers

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def make_acc_image(acc_grid):
    
    acc_image = np.zeros((256,256))
    
    # Bottom row
    acc_image[:31,:31] = acc_grid[0]
    acc_image[:31,32:63] = acc_grid[1]
    acc_image[:31,64:95] = acc_grid[2]
    acc_image[:31,96:127] = acc_grid[3]
    acc_image[:31,128:159] = acc_grid[4]
    acc_image[:31,160:191] = acc_grid[5]
    acc_image[:31,192:223] = acc_grid[6]
    acc_image[:31,224:255] = acc_grid[7]
    
    # Next row = 2
    acc_image[32:63,:31] = acc_grid[8]
    acc_image[32:63,32:63] = acc_grid[9]
    acc_image[32:63,64:95] = acc_grid[10]
    acc_image[32:63,96:127] = acc_grid[11]
    acc_image[32:63,128:159] = acc_grid[12]
    acc_image[32:63,160:191] = acc_grid[13]
    acc_image[32:63,192:223] = acc_grid[14]
    acc_image[32:63,224:255] = acc_grid[15]
 
    # Next row = 3
    acc_image[64:95,:31] = acc_grid[16]
    acc_image[64:95,32:63] = acc_grid[17]
    acc_image[64:95,64:95] = acc_grid[18]
    acc_image[64:95,96:127] = acc_grid[19]
    acc_image[64:95,128:159] = acc_grid[20]
    acc_image[64:95,160:191] = acc_grid[21]
    acc_image[64:95,192:223] = acc_grid[22]
    acc_image[64:95,224:255] = acc_grid[23]
    
    
    # Next row = 4
    acc_image[96:127,:31] = acc_grid[24]
    acc_image[96:127,32:63] = acc_grid[25]
    acc_image[96:127,64:95] = acc_grid[26]
    acc_image[96:127,96:127] = acc_grid[27]
    acc_image[96:127,128:159] = acc_grid[28]
    acc_image[96:127,160:191] = acc_grid[29]
    acc_image[96:127,192:223] = acc_grid[30]
    acc_image[96:127,224:255] = acc_grid[31]
    
    # Next row = 5
    acc_image[128:159,:31] = acc_grid[32]
    acc_image[128:159,32:63] = acc_grid[33]
    acc_image[128:159,64:95] = acc_grid[34]
    acc_image[128:159,96:127] = acc_grid[35]
    acc_image[128:159,128:159] = acc_grid[36]
    acc_image[128:159,160:191] = acc_grid[37]
    acc_image[128:159,192:223] = acc_grid[38]
    acc_image[128:159,224:255] = acc_grid[39]
    
     # Next row = 6
    acc_image[160:191,:31] = acc_grid[40]
    acc_image[160:191,32:63] = acc_grid[41]
    acc_image[160:191,64:95] = acc_grid[42]
    acc_image[160:191,96:127] = acc_grid[43]
    acc_image[160:191,128:159] = acc_grid[44]
    acc_image[160:191,160:191] = acc_grid[45]
    acc_image[160:191,192:223] = acc_grid[46]
    acc_image[160:191,224:255] = acc_grid[47]
   
     # Next row = 7
    acc_image[192:223,:31] = acc_grid[48]
    acc_image[192:223,32:63] = acc_grid[49]
    acc_image[192:223,64:95] = acc_grid[50]
    acc_image[192:223,96:127] = acc_grid[51]
    acc_image[192:223,128:159] = acc_grid[52]
    acc_image[192:223,160:191] = acc_grid[53]
    acc_image[192:223,192:223] = acc_grid[54]
    acc_image[192:223,224:255] = acc_grid[55]
    
    
     # Next row = 8
    acc_image[224:255,:31] = acc_grid[56]
    acc_image[224:255,32:63] = acc_grid[57]
    acc_image[224:255,64:95] = acc_grid[58]
    acc_image[224:255,96:127] = acc_grid[59]
    acc_image[224:255,128:159] = acc_grid[60]
    acc_image[224:255,160:191] = acc_grid[61]
    acc_image[224:255,192:223] = acc_grid[62]
    acc_image[224:255,224:255] = acc_grid[63]
     
    return acc_image

In [ ]:
# Set the paths to the raw data files
whereami = '/home/jhargis'
#whereami = '/Users/jhargis'
path     = 'Dropbox/astroNN/wl_maps/'
#whereami = '/Users/goldston'
#whereami = '/Users/jegpeek'
#path = 'Documents/Weak_Lensing/kmaps_smoothed/'


In [ ]:
# Set (1) the factor by which we want to degrade the original WL maps
# and (2) the number of realizations of each universe.
#
#   The original images are 2048 x 2048, and we degrade them using
#   an 8 x 8 sq.pix box, which makes a smaller set of 64 images 
#   (= 256x256 sq.pix in size).

degrade=8
nct = 9

data, labels = read_WL(path,display=True)

print "Data shape :",data.shape
print "Labels     :",labels

In [ ]:
# Display an example of the full 2048 x 2048 image
fullimage = getFITS("smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.850_4096xy_0009r_0029p_0100z_og.gre.fit")
plt.figure(figsize=(12, 6))
plt.imshow(fullimage, origin="lower")
plt.clim(-.015,0.025)
plt.colorbar()

In [ ]:
# Now compare a small section of the original image to the rebinned version
#   Becaused we used an 8x8 box, a 256x256 region in the original image
#   should correspond to a 32x32 region in the rebinned image
plt.figure(figsize=(12, 4))
plt.imshow(fullimage[:256,:256],origin="lower")
plt.clim(-0.015,0.025)
plt.colorbar()
plt.show()

image_data = data[:32,:32,8,1]
image_data.shape
plt.figure(figsize=(12, 4))
plt.imshow(image_data, origin='lower')
plt.clim(-0.015,0.025)
plt.colorbar()
plt.show()

In [ ]:
# First we slice the data in the following manner:
#
#   1) Each 256 x 256 image is again split 8 x 8 into 64 images which are 32 x 32 sq. pix in size
#   2) Series of 4 rotations, 2 flips, and shifts of 2 pixels in size
imgs2, x_centers, y_centers = slice_data(data, labels, 3, 3)
img2sh = imgs2.shape

# Next, reshape the arrays and take the first 7 realizations as the training data set
train_dataset = np.transpose(imgs2[:, :, 0:7, :, :].reshape(img2sh[0], img2sh[1], 7.0*img2sh[3]*2.0), (2, 0, 1))
train_xc = x_centers[0:7, :, :].reshape(7.0*img2sh[3]*2.0)
train_yc = y_centers[0:7, :, :].reshape(7.0*img2sh[3]*2.0)
ones = np.ones([7,img2sh[3], 2] )
train_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(7.0*img2sh[3]*2.0)

# The validation set is the 8th realization
valid_dataset = np.transpose(imgs2[:, :, 7, :, :].reshape(img2sh[0], img2sh[1], 1.0*img2sh[3]*2.0), (2, 0, 1))
valid_xc = x_centers[7, :, :].reshape(1.0*img2sh[3]*2.0)
valid_yc = y_centers[7, :, :].reshape(1.0*img2sh[3]*2.0)
ones = np.ones([1,img2sh[3], 2] )
valid_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(1.0*img2sh[3]*2.0)

# The test data set is the 9th realization
test_dataset = np.transpose(imgs2[:, :, 8, :, :].reshape(img2sh[0], img2sh[1], 1.0*img2sh[3]*2.0), (2, 0, 1))
test_xc = x_centers[8, :, :].reshape(1.0*img2sh[3]*2.0)
test_yc = y_centers[8, :, :].reshape(1.0*img2sh[3]*2.0)
ones = np.ones([1,img2sh[3], 2] )
test_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(1.0*img2sh[3]*2.0)


In [ ]:
print "Master images tensor shape:", img2sh
print
print "Train dataset shape:", train_dataset.shape
print "Train labels shape :", train_labels.shape
print "Test dataset shape :", test_dataset.shape
print "Valid dataset shape:", valid_dataset.shape
print "TOTAL data sets    : ", train_dataset.shape[0] + test_dataset.shape[0] + valid_dataset.shape[0]

In [ ]:
plt.plot(np.reshape(x_centers, 9*19720*2), np.reshape(y_centers, 9*19720*2), '.')

In [ ]:
fig = plt.figure(figsize=(15, 7))
sub1 = plt.subplot(2, 6, 1)
sub1.set_xticks(())
sub1.set_yticks(())
sub1.imshow(image_data, origin='lower')

image_data = test_dataset[0,:32,:32]
sub2 = plt.subplot(2, 6, 2)
sub2.set_xticks(())
sub2.set_yticks(())
sub2.imshow(image_data, origin='lower')


In [ ]:
# Loop to get array elements for grid
indv = np.arange(1,1024,2)
k = 0
test_grid = []
for i in range(256):
    test_grid.append(indv[k:k+8])
    k+=8

# Plot the full figure (rebinned)
plt.figure(figsize=(12, 6))
image_data = data[:,:,8,1]
#plt.imshow(fullimage, origin="lower")
plt.imshow(image_data, origin="lower")
plt.clim(-.015,0.025)
plt.colorbar()


# Plot based on setup grid
fig, axes = plt.subplots(nrows=1, ncols=8,figsize=(10,10))
gridpt = 0
j = 0
print "test_grid:",test_grid[gridpt]
for ax in axes.flat:
    idx = test_grid[gridpt][j]
    pldata = test_dataset[idx, :32, :32]
    im = ax.imshow(pldata, origin='lower',vmin=-0.015, vmax=0.025)
    ax.set_xticks(())
    ax.set_yticks(())
    j+=1    
fig.subplots_adjust(right=1.)
#cbar_ax = fig.add_axes(([0.85, 0.15, 0.05, 0.7]))
#fig.colorbar(im, cax=cbar_ax)

 
# Plot based on setup grid
fig, axes = plt.subplots(nrows=1, ncols=8,figsize=(10,10))
gridpt = 63
j = 0
print "test_grid:",test_grid[gridpt]
for ax in axes.flat:
    idx = test_grid[gridpt][j]
    pldata = test_dataset[idx, :32, :32]
    im = ax.imshow(pldata, origin='lower',vmin=-0.015, vmax=0.025)
    ax.set_xticks(())
    ax.set_yticks(())
    j+=1    
fig.subplots_adjust(right=1.)
#cbar_ax = fig.add_axes(([0.85, 0.15, 0.05, 0.7]))
#fig.colorbar(im, cax=cbar_ax)

       

In [ ]:
# Just don't even worry about this for now.
# Figure it out later.

"""
#pickle_file = 'notMNIST.pickle'
#pickle_file = '/Users/jegpeek/Documents/WL88.pickle'
pickle_file = '/Users/jegpeek/Dropbox/WL_other.pickle'

usePickle = True

if usePickle:
    with open(pickle_file, 'rb') as f:
      save = pickle.load(f)
      train_dataset = save['train_dataset']
      train_labels = save['train_labels']
      valid_dataset = save['valid_dataset']
      valid_labels = save['valid_labels']
      test_dataset = save['test_dataset']
      test_labels = save['test_labels']
      del save  # hint to help gc free up memory
      print('Training set', train_dataset.shape, train_labels.shape)
      print('Validation set', valid_dataset.shape, valid_labels.shape)
      print('Test set', test_dataset.shape, test_labels.shape)
else:
    %run Read_WL.py
"""

In [ ]:
# Reformat into a TensorFlow-friendly shape:
# - convolutions need the image data formatted as a cube (width by height by #channels)
# - labels as float 1-hot encodings.

image_size = 32
num_labels = 2
num_channels = 1 # grayscale

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
print "test_labels:", test_labels[test_grid[0]]

In [ ]:
# Cut this down to a subset
test_frac = 0.3
_train_dataset = train_dataset[0:train_dataset.shape[0]*test_frac,:,:,:]
_train_labels  = train_labels[0:train_labels.shape[0]*test_frac,:]
_test_dataset = test_dataset[0:test_dataset.shape[0]*test_frac,:,:,:]
_test_labels  = test_labels[0:test_labels.shape[0]*test_frac,:]
_valid_dataset = valid_dataset[0:valid_dataset.shape[0]*test_frac,:,:,:]
_valid_labels  = valid_labels[0:valid_labels.shape[0]*test_frac,:]
print('--> Using subset <--')
#print('Training set', _train_dataset.shape, _train_labels.shape)
#print('Validation set', _valid_dataset.shape, _valid_labels.shape)
#print('Test set', _test_dataset.shape, _test_labels.shape)
train_dataset = _train_dataset
train_labels = _train_labels
test_dataset = _test_dataset
test_labels = _test_labels
valid_dataset = _valid_dataset
valid_labels = _valid_labels
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
batch_size = 128  # 16
patch_size = 5    # 5
depth = 32        # 16
depth2= 64        # JRH addition
num_hidden = 2048   # 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Global Step
    #global_step = tf.Variable(1.)
    #learn_decay = 0.85
    #learning_rate = tf.train.exponential_decay(0.005, global_step, 10000, learn_decay, staircase=True)
    learning_rate = 0.01

    # Variables.
    #layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    #layer1_biases = tf.Variable(tf.zeros([depth]))
    #layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    #layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    #layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    #layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    #layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    #layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Alternate variable setup
    #   Layer 1: Compute 16 features = depth
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    #layer1_biases = tf.Variable(tf.zeros([depth]))
    layer1_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
    
    #   Layer 2: Compute 32 features = DEPTH2
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth2]))
    
    #   Layer 3: Fully-connected layer should use depth2, which results from layer2
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
    
    #   Layer 4: Readout layer
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))

    keep_prob = tf.placeholder(tf.float32)
    
    #with tf.name_scope('derp'):
    #  spl = tf.split(3, 16, layer1_weights)
    #  filter_summary = tf.image_summary((spl[0]).name, spl[0], max_images=1)

    # Model.
    def model(data):
        #  Layer 1
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        #  Layer 2
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        #  Layer 3 = fully connected
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        #  Layer 4 = readout layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    def variable_summaries(var, name):
        with tf.name_scope("summaries"):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)
        
    with tf.name_scope('layer1'):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            variable_summaries(layer1_weights, 'layer1/weights')
            with tf.name_scope("images"):
                spl = tf.split(3, 8, layer1_weights)
                tf.image_summary("layer1/weights", spl[0])
        with tf.name_scope("biases"):
            variable_summaries(layer1_biases, 'layer1/biases')

    with tf.name_scope('layer2'):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            variable_summaries(layer2_weights, 'layer2/weights')
            with tf.name_scope("images"):
                spl = tf.split(3, 16, layer2_weights)
                tf.image_summary("layer2/weights", spl[0])
        with tf.name_scope("biases"):
            variable_summaries(layer2_biases, 'layer2/biases')

    with tf.name_scope('layer3'):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            variable_summaries(layer2_weights, 'layer3/weights')
        with tf.name_scope("biases"):
            variable_summaries(layer3_biases, 'layer3/biases')
            
    with tf.name_scope('layer4'):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            variable_summaries(layer4_weights, 'layer4/weights')
        with tf.name_scope("biases"):
            variable_summaries(layer4_biases, 'layer4/biases')
            
                
    with tf.name_scope('loss'):
        # Try to log the loss
        tf.scalar_summary('loss', loss)
        
    with tf.name_scope('learning_rate'):
        tf.scalar_summary('learning_rate', learning_rate)
                
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

    merged = tf.merge_all_summaries()

In [ ]:
valid_prediction.get_shape()

In [ ]:
permutation = np.random.permutation(train_labels.shape[0])
train_dataset = train_dataset[permutation,:,:]
train_labels = train_labels[permutation]

In [ ]:
# Run of TensorFlow without the permutations
# but with the TensorBoard logging calls
num_steps = 200001
print_step = 200
summary_step = 200
losses = np.zeros((num_steps-1)/print_step+1)
acc_valid = np.zeros((num_steps-1)/print_step+1)
acc_test = np.zeros((num_steps-1)/print_step+1)
acc_train = np.zeros((num_steps-1)/print_step+1)
acc_grid = np.zeros(64)
acc_image = np.zeros((256,256))
q = 0
p = 0

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    summary_writer = tf.train.SummaryWriter(whereami+'/Documents/logs', session.graph_def)
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        start_time = time.time()
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summary, _, l, predictions = session.run([merged, optimizer, loss, train_prediction], feed_dict=feed_dict)
        duration = time.time() - start_time
        if (step % print_step == 0):
            losses[q] = l
            test_prediction_results = test_prediction.eval()
            acc_valid[q] = accuracy(valid_prediction.eval(), valid_labels)/100.0
            acc_test[q] = accuracy(test_prediction_results, test_labels)/100.0
            acc_train[q] = accuracy(predictions, batch_labels)/100.0
            for i in range(len(acc_grid)):
                acc_grid[i] = accuracy(test_prediction_results[test_grid[i]], test_labels[test_grid[i]])/100.0
            q += 1
            
            # Setup the plot
            #fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(10,7))
            fig = plt.figure(figsize=(10,7))
            ax1 = plt.subplot2grid((2,2),(0,0),colspan=2)
            ax2 = plt.subplot2grid((2,2),(1,0))
            ax3 = plt.subplot2grid((2,2),(1,1))
            
            ax1.plot(np.arange(0,(num_steps-1)/print_step+1), acc_valid, '.', color='b',alpha=0.5)
            ax1.plot((-1)*np.arange(0,(num_steps-1)/print_step+1),acc_test, '.', color='g',alpha=0.5)
            ax1.plot(np.arange(0,(num_steps-1)/print_step+1), acc_train, '.', color='r',alpha=0.5)
            ax1.set_ylim([0.45, 1.05])
            #plt.xlim([-1000, 1000])
            ax1.set_xlim([-1*((step/print_step)*1.1),(step/print_step)*1.1])
            #plt.xlim([-1*(((num_steps-1)/print_step+1)*1.1),((num_steps-1)/print_step+1)*1.1])
            
            # Plot the accuracy grid
            acc_image = make_acc_image(acc_grid)
            #fig = plt.figure(figsize=(12, 6))
            #sub1 = plt.subplot(1, 2, 1)
            ax2.set_xticks(())
            ax2.set_yticks(())
            im2 = ax2.imshow(acc_image, origin='lower',vmin = 0, vmax = 1.0)
            divider2 = make_axes_locatable(ax2)
            cim2 = divider2.append_axes("right", size="20%", pad=0.5)
            cbar2 = plt.colorbar(im2, cax=cim2, ticks=MultipleLocator(0.2), format="%.2f")
            #sub2 = plt.subplot(1, 2, 2)
            ax3.set_xticks(())
            ax3.set_yticks(())
            im3 = ax3.imshow(image_data, origin='lower',vmin = -0.015, vmax = 0.025)
            divider3 = make_axes_locatable(ax3)
            cim3 = divider3.append_axes("right", size="20%", pad=0.5)
            cbar3 = plt.colorbar(im3, cax=cim3, ticks=MultipleLocator(0.2))

            
            display.clear_output(wait=True)
            display.display(plt.gcf())
            print('Minibatch loss at step %d: %f (%.3f sec)' % (step, l, duration))
            #print('  Grid point acc:',acc_grid[0], acc_grid[1], acc_grid[2], acc_grid[3], acc_grid[4], acc_grid[5], acc_grid[6], acc_grid[7])
            #print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            #print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
        if (step % summary_step == 0):
            #print "got here"
            summary_writer.add_summary(summary, p)
            p += 1
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [ ]:
# permutation = np.random.permutation(train_labels.shape[0])
#train_dataset = train_dataset[permutation,:,:]
#train_labels = train_labels[permutation]

In [ ]:
test_prediction_results[test_grid[0]]

In [ ]:
test_labels[test_grid[0]]

In [ ]:
accuracy(test_prediction_results[test_grid[0]],test_labels[test_grid[0]])/100.
acc_grid

num_steps = 20001
print_step = 200
losses = np.zeros((num_steps-1)/print_step+1)
acc_valid = np.zeros((num_steps-1)/print_step+1)
acc_test = np.zeros((num_steps-1)/print_step+1)
q = 0
p=0

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % print_step == 0):
            losses[q] = l
            acc_valid[q] = accuracy(valid_prediction.eval(), valid_labels)/100.0
            acc_test[q] = accuracy(test_prediction.eval(), test_labels)/100.0
            q += 1
            plt.plot(np.arange(0,(num_steps-1)/print_step+1), acc_valid, '.', color='b')
            plt.plot((-1)*np.arange(0,(num_steps-1)/print_step+1),acc_test, '.', color='g')
            plt.ylim([0.45, 0.65])
            plt.xlim([-1000, 1000])
            display.clear_output(wait=True)
            display.display(plt.gcf())
            print('Minibatch loss at step %d: %f' % (step, l))
            #print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            #print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

plt.plot(np.arange(0,(num_steps-1)/print_step+1), acc_valid, '.', color='b')
plt.plot((-1)*np.arange(0,(num_steps-1)/print_step+1),acc_test, '.', color='g')
plt.ylim([0.45, 0.75])
plt.xlim([-100, 100])

In [ ]:
plt.plot(acc_test, acc_valid, '.', color='b')
#plt.plot((-1)*np.arange(0,(num_steps-1)/print_step+1),losses, '.', color='g')
plt.ylim([0.4, 0.7])
plt.xlim([0.0, 1.0])

**Open questions:**
+ Why is there so much scatter in the loss function over time?
+ Is there structure in the loss function over time?
+ If I plot loss vs. accuracy, what do I get? 
+ Do I really see a difference when I scramble vs. leave in order, and if so, is it because of the way SGD interacts with the two cosmologies?
+ will deeper / better networks get us over 65%?
+ Why, oh why, are my test and valid data sets so damn well correlated??

graph.get_tensor_by_name.im_func

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---